In [72]:
from google.cloud import bigquery
import os
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import datetime
from google.cloud import bigquery
import pytz
from google.oauth2 import service_account
import numpy as np
import requests
from datetime import datetime as dt
from datetime import date


In [73]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'.\creds\opendatadbt-f2e674527eb6.json'
client = bigquery.Client()

credentials = ServiceAccountCredentials.from_json_keyfile_name(r'.\creds\opendatadbt-f2e674527eb6.json') # Your json file here



#Initial load 
littlerock_table_id = "opendatadbt.311.littlerock311"

littlerock311 = requests.get("https://data.littlerock.gov/resource/2x6n-j9fb.json?$limit=10000&$where=ticket_created_date_time>='2021-03-22T00:00:00.000'") 
littlerockjson = littlerock311.json()
littlerock_df_parsed = littlerock_df['geocoded_column'].apply(pd.Series)
littlerock_df_parsed = littlerock_df_parsed.drop(littlerock_df_parsed.columns[[0,1]], axis=1)
littlerock_df_cleaned = pd.concat([littlerock_df.drop(['geocoded_column'], axis=1), littlerock_df_parsed], axis=1)
littlerock_df_cleaned
cols = ['ticket_id','ticket_created_date_time','ticket_closed_date_time','issue_type','issue_sub_category','human_address','ticket_status','latitude','longitude']
littlerock_df_cleaned = pd.DataFrame(littlerock_df_cleaned, columns = cols)

In [74]:
#Incremental load
littlerock_table_id = "opendatadbt.311.littlerock311"
current_datetime = dt.today().strftime("%Y-%m-%dT%H:%M:%S.000")
query_string = """
SELECT 
max(ticket_created_date_time), max(ticket_closed_date_time)
FROM `opendatadbt.311.littlerock311`
limit 10
"""

littlerockmaxopendatedataframe = (
    client.query(query_string)
    .result()
    .to_dataframe()
)
littlerockmaxopendatedate = str(littlerockmaxopendatedataframe['f0_'][0])

littlerockmaxclosedatedataframe = (
    client.query(query_string)
    .result()
    .to_dataframe()
)
littlerockmaxclosedate = str(littlerockmaxclosedatedataframe['f1_'][0])

In [75]:

littlerock_table_id = "opendatadbt.311.littlerock311"

littlerock311 = requests.get("https://data.littlerock.gov/resource/2x6n-j9fb.json?$limit=10000&$where=ticket_created_date_time>'"+littlerockmaxopendatedate+"' OR ticket_closed_date_time between '" +littlerockmaxclosedate+ "' AND '" + current_datetime + "'") 
littlerockjson = littlerock311.json()
littlerock_df_parsed = littlerock_df['geocoded_column'].apply(pd.Series)
littlerock_df_parsed = littlerock_df_parsed.drop(littlerock_df_parsed.columns[[0,1]], axis=1)
littlerock_df_cleaned = pd.concat([littlerock_df.drop(['geocoded_column'], axis=1), littlerock_df_parsed], axis=1)
littlerock_df_cleaned
cols = ['ticket_id','ticket_created_date_time','ticket_closed_date_time','issue_type','issue_sub_category','human_address','ticket_status','latitude','longitude']
littlerock_df_cleaned = pd.DataFrame(littlerock_df_cleaned, columns = cols)

In [76]:
littlerock_df_cleaned

,ticket_id,ticket_created_date_time,ticket_closed_date_time,issue_type,issue_sub_category,human_address,ticket_status,latitude,longitude
0,21-00025000,2021-03-22T06:46:00.000,2021-03-22T13:09:00.000,Trash Complaints,Missed Garbage Pick-Up,"{""address"": ""5620 C ST"", ""city"": ""LITTLE ROCK""...",Open,34.754748,-92.33714646
1,21-00025001,2021-03-22T06:49:00.000,2021-03-22T06:49:00.000,Street Lights,Street Light - Out,"{""address"": ""2510 SILVER MAPLE DR"", ""city"": ""L...",Closed,34.66453059,-92.43897046
2,21-00025003,2021-03-22T07:04:00.000,2021-03-22T13:09:00.000,Trash Complaints,Missed Garbage Pick-Up,"{""address"": ""18109 FAWN TREE DR"", ""city"": ""LIT...",Open,34.66196738,-92.43746741
3,21-00025004,2021-03-22T07:05:00.000,2021-03-22T14:02:00.000,Trash Complaints,Knuckleboom Pick-Up,"{""address"": ""6 ARNOLD PALMER CV"", ""city"": ""LIT...",Closed,34.6690911,-92.43258649
4,21-00025005,2021-03-22T07:11:00.000,2021-03-22T07:11:00.000,Trash Complaints,Garbage Cart Issues,"{""address"": ""10915 APPOMATTOX DR"", ""city"": ""LI...",Open,34.65068968,-92.3724362
...,...,...,...,...,...,...,...,...,...
485,21-00025737,2021-03-22T21:21:00.000,2021-03-22T21:21:00.000,Public Works,Ditch Maintenance,"{""address"": ""5101 LOETSCHER LN"", ""city"": ""LITT...",Open,34.66203865,-92.33510181
486,21-00025738,2021-03-22T22:05:00.000,2021-03-22T22:05:00.000,Public Works,Pothole Repair,"{""address"": ""901 WELLINGTON VILLAGE RD"", ""city...",Open,34.76218288,-92.44056715
487,21-00025739,2021-03-22T22:08:00.000,2021-03-22T22:08:00.000,Public Works,Pothole Repair,"{""address"": ""15509 CHENAL PKY"", ""city"": ""LITTL...",Open,34.75564012,-92.4399619
488,21-00025741,2021-03-23T01:37:00.000,2021-03-23T01:37:00.000,Code Violations,Trash / Rubbish / Debris on Premise,"{""address"": ""37 PURDUE CIR"", ""city"": ""LITTLE R...",Open,34.72004256,-92.34538147


In [77]:
littlerock_table_id = "opendatadbt.311.littlerock311"
dataset_ref = client.dataset('311')
table_ref = dataset_ref.table('littlerock311')
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",
)

client.load_table_from_dataframe(
    littlerock_df_cleaned, table_ref, job_config=job_config
).result()  # Make an API request.
